@Thomas, you can use `get_beh2_circuits_to_compile()`, and `get_nah_circuits_to_compile()` `get_h2o_circuits_to_compile()`.

These functions return a `circuits_to_compile` list. Each item in the list is a tuple, `(circuit, connected_qubit_pairs)`. The first few elements of this list for BeH2 are printed in the last cell of this notebook.

You'll want to iterate over the `circuits_to_compile`, compile each circuit, and report back the time it took to compile the circuit. We'll do something similar for full QOC (except the list will be a little different since full QOC doesn't divide into one-angle slices).

In [35]:
import random
import numpy as np
import tensorflow as tf
import sys
sys.path.append('..')
from fqc import uccsd, util, qaoa
from qiskit import QuantumCircuit, QuantumRegister
from copy import deepcopy

In [20]:
def set_seeds():
    random.seed(0)
    np.random.seed(1)
    tf.set_random_seed(2)

In [21]:
def _get_slice_circuits_list(circuit, blockings):
    set_seeds()

    def indices(gate):
        return [qarg[1] for qarg in gate.qargs]

    def gate_block_index(gate, blocking):
        if len(indices(gate)) == 1:
            return [indices(gate)[0] in grouping for grouping in blocking].index(True)
        else:
            # check which block each qubit is in
            control_index, target_index = indices(gate)
            control_block_index = [control_index in grouping for grouping in blocking].index(True)
            target_block_index = [target_index in grouping for grouping in blocking].index(True)
            if control_block_index != target_block_index:
                return -1
            else:
                return control_block_index

    blockings_index = 0

    gates = circuit.data
    width = circuit.width()

    slice_circuits_list = []

    while len(gates) > 0:
        blocking = blockings[blockings_index]
        remaining_gates = []; contaminated_indices = set()

        slice_circuits = [deepcopy(circuit) for _ in range(len(blocking.blocks))]
        for slice_circuit in slice_circuits:
            slice_circuit.data = []

        for i, gate in enumerate(gates):
            if len(contaminated_indices) == width:
                remaining_gates.extend(gates[i:])
                break

            block_index = gate_block_index(gate, blocking.blocks)
            if block_index == -1:
                contaminated_indices.add(indices(gate)[0]); contaminated_indices.add(indices(gate)[1]);
                remaining_gates.append(gate)
            else:
                if len(indices(gate)) == 1:
                    if indices(gate)[0] in contaminated_indices:
                        remaining_gates.append(gate)
                    else:
                        slice_circuits[block_index].data.append(gate)
                else:
                    if indices(gate)[0] in contaminated_indices or indices(gate)[1] in contaminated_indices:
                        contaminated_indices.add(indices(gate)[0]); contaminated_indices.add(indices(gate)[1]);
                        remaining_gates.append(gate)
                    else:
                        slice_circuits[block_index].data.append(gate)

        slice_circuits_list.append((slice_circuits, blocking))
        gates = remaining_gates
        blockings_index = (blockings_index + 1) % len(blockings)


    return slice_circuits_list


def _get_circuits_to_compile(slice_circuits_list, sampling_rate=None):
    circuits_to_compile = []
    for slice_circuits, blocking in slice_circuits_list:
        if sampling_rate is not None:
            if random.random() > sampling_rate:
                continue

        for slice_circuit, block, connected_qubit_pairs in zip(
            slice_circuits, blocking.blocks, blocking.connected_qubit_pairs_list):
            for index in block:
                assert len(slice_circuit.qregs) == 1
                slice_circuit.iden(slice_circuit.qregs[0][index])
            slice_circuit = util.squash_circuit(slice_circuit)
            
            for subslice in uccsd.get_uccsd_slices(slice_circuit, granularity=2, dependence_grouping=True):
                circuits_to_compile.append((subslice.circuit, connected_qubit_pairs))

    return circuits_to_compile

In [22]:
def get_beh2_circuits_to_compile(sampling_rate=None):
    set_seeds()
    
    circuit = uccsd.get_uccsd_circuit('BeH2')
    circuit = util.circuitutil.optimize_circuit(circuit)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 3)
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

    # layout is 0 2 4
    #           1 3 5
    
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]

    class Blocking2(object):
        blocks = [{0, 1}, {2, 3, 4, 5}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)]]

    blockings = [Blocking1, Blocking2]
    
    slice_circuits_list = _get_slice_circuits_list(circuit, blockings)
    return _get_circuits_to_compile(slice_circuits_list, sampling_rate=sampling_rate)

In [23]:
def get_nah_circuits_to_compile(sampling_rate=None):
    set_seeds()
    
    circuit = uccsd.get_uccsd_circuit('NaH')
    circuit = util.circuitutil.optimize_circuit(circuit)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 4)
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

    # layout is 0 2 4 6
    #           1 3 5 7
    
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5, 6, 7}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1), (1, 3), (2, 3), (0, 2)]]

    class Blocking2(object):
        blocks = [{0, 1}, {2, 3, 4, 5}, {6, 7}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]

    blockings = [Blocking1, Blocking2]
    
    slice_circuits_list = _get_slice_circuits_list(circuit, blockings)
    return _get_circuits_to_compile(slice_circuits_list, sampling_rate=sampling_rate)

In [24]:
def get_h2o_circuits_to_compile(sampling_rate=None):
    set_seeds()
    
    circuit = uccsd.get_uccsd_circuit('H2O')
    circuit = util.circuitutil.optimize_circuit(circuit)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 5)
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

    # layout is 0 2 4 6 8
    #           1 3 5 7 9
    
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5, 6, 7}, {8, 9}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]

    class Blocking2(object):
        blocks = [{0, 1}, {2, 3, 4, 5}, {6, 7, 8, 9}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1), (1, 3), (2, 3), (0, 2)]]

    blockings = [Blocking1, Blocking2]
    
    slice_circuits_list = _get_slice_circuits_list(circuit, blockings)
    return _get_circuits_to_compile(slice_circuits_list, sampling_rate=sampling_rate)

In [55]:
def _get_qaoa_circuits_to_compile(graph_type, graph_N, graph_p, blockings, sampling_rate=None):
    assert graph_p in [1, 2, 3, 4, 5, 6, 7, 8], 'we only did p = 1...8'
    set_seeds()
    circuit = qaoa.get_qaoa_circuit(graph_N, graph_p, graph_type)
    circuit = util.circuitutil.optimize_circuit(circuit)
    coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, int(graph_N/2))
    circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
    slice_circuits_list = _get_slice_circuits_list(circuit, blockings)
    return _get_circuits_to_compile(slice_circuits_list, sampling_rate=sampling_rate)

def get_N6_erdosrenyi_qaoa_circuits_to_compile(graph_p, sampling_rate=None):
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]

    class Blocking2(object):
        blocks = [{0, 2, 4}, {1, 3, 5}]
        connected_qubit_pairs_list = [[(0, 1), (1, 2)], [(0, 1), (1, 2)]]    

    class Blocking3(object):
        blocks = [{0, 1}, {2, 3, 4, 5}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)]]
 
    blockings = [Blocking1, Blocking2, Blocking3]
    return _get_qaoa_circuits_to_compile('ErdosRenyi', 6, graph_p, blockings, sampling_rate)


def get_N8_erdosrenyi_qaoa_circuits_to_compile(graph_p, sampling_rate=None):
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5, 6, 7}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)],[(0, 1), (1, 3), (2, 3), (0, 2)]]

    class Blocking2(object):
        blocks = [{0, 2, 4, 6}, {1, 3, 5, 7}]
        connected_qubit_pairs_list = [[(0, 1), (1, 2), (2, 3)], [(0, 1), (1, 2), (2, 3)]]    

    class Blocking3(object):
        blocks = [{0, 1}, {2, 3, 4, 5}, {6, 7}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]
 
    blockings = [Blocking1, Blocking2, Blocking3]
    return _get_qaoa_circuits_to_compile('ErdosRenyi', 8, graph_p, blockings, sampling_rate)


def get_N6_3reg_qaoa_circuits_to_compile(graph_p, sampling_rate=None):
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]

    class Blocking2(object):
        blocks = [{0, 2, 4}, {1, 3, 5}]
        connected_qubit_pairs_list = [[(0, 1), (1, 2)], [(0, 1), (1, 2)]]    

    class Blocking3(object):
        blocks = [{0, 1}, {2, 3, 4, 5}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)]]
 
    blockings = [Blocking1, Blocking2, Blocking3]
    return _get_qaoa_circuits_to_compile('3Reg', 6, graph_p, blockings, sampling_rate)


def get_N8_3reg_qaoa_circuits_to_compile(graph_p, sampling_rate=None):
    class Blocking1(object):
        blocks = [{0, 1, 2, 3}, {4, 5, 6, 7}]
        connected_qubit_pairs_list = [[(0, 1), (1, 3), (2, 3), (0, 2)],[(0, 1), (1, 3), (2, 3), (0, 2)]]

    class Blocking2(object):
        blocks = [{0, 2, 4, 6}, {1, 3, 5, 7}]
        connected_qubit_pairs_list = [[(0, 1), (1, 2), (2, 3)], [(0, 1), (1, 2), (2, 3)]]    

    class Blocking3(object):
        blocks = [{0, 1}, {2, 3, 4, 5}, {6, 7}]
        connected_qubit_pairs_list = [[(0, 1)], [(0, 1), (1, 3), (2, 3), (0, 2)], [(0, 1)]]
 
    blockings = [Blocking1, Blocking2, Blocking3]
    return _get_qaoa_circuits_to_compile('3Reg', 8, graph_p, blockings, sampling_rate)

In [25]:
circuits_to_compile = get_beh2_circuits_to_compile()
len(circuits_to_compile)

494

In [28]:
circuits_to_compile = get_beh2_circuits_to_compile(sampling_rate=0.1)
len(circuits_to_compile)

40

In [59]:
# p = 8 examples.
circuits_to_compile = get_N6_erdosrenyi_qaoa_circuits_to_compile(8, sampling_rate=0.5)
print(len(circuits_to_compile))

circuits_to_compile = get_N8_erdosrenyi_qaoa_circuits_to_compile(8, sampling_rate=0.5)
print(len(circuits_to_compile))

circuits_to_compile = get_N6_3reg_qaoa_circuits_to_compile(8, sampling_rate=0.5)
print(len(circuits_to_compile))

circuits_to_compile = get_N8_3reg_qaoa_circuits_to_compile(8, sampling_rate=0.5)
print(len(circuits_to_compile))

21
55
26
73


In [29]:
for circuit, connected_qubit_pairs in circuits_to_compile[:10]:
    print(circuit)
    print(connected_qubit_pairs)
    print('\n\n\n--------------------------------------------------------------------------------\n\n\n')

                  ┌────┐
q1924_0: |0>──────┤ Id ├
            ┌────┐└────┘
q1924_1: |0>┤ Id ├──────
            └────┘      
[(0, 1)]



--------------------------------------------------------------------------------



                                                                            »
q1925_0: |0>────────■───────────────────────────────────────────────────────»
            ┌────┐  │                                                       »
q1925_1: |0>┤ Id ├──┼───────────────────────────────────────────────────────»
            └────┘┌─┴─┐                                                     »
q1925_2: |0>──────┤ X ├──■───────────────────────────────────────────────■──»
                  └───┘┌─┴─┐┌────────────┐┌─────────────┐┌────────────┐┌─┴─┐»
q1925_3: |0>───────────┤ X ├┤ Rx(3.1416) ├┤ Rz(0.65847) ├┤ Rx(3.1416) ├┤ X ├»
                       └───┘└────────────┘└─────────────┘└────────────┘└───┘»
«                    ┌────┐                         
«q1925_0: ────────■──┤ I